In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import *

In [2]:
spark=SparkSession.builder.appName("Data Analysis").getOrCreate()

In [34]:
df_playstore = spark.read.csv("C:/Users/ABT/Desktop/JOB_HUNT_2024/portfolio_github/RealTime/Data/googleplaystore.csv",inferSchema=True,header=True)

In [35]:
df_playstore.printSchema()
df_playstore.show(1,False)

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

+----------------------------------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|App                                           |Category      |Rating|Reviews|Size|Installs|Type|Price|Content Rating|Genres      |Last Updated   |Current Ver|Android Ver |
+----------------------------------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+--

# Data Cleaning

In [36]:
df_playstore=df_playstore.drop("size","Content Rating","Last Updated","Android Ver","Current Ver")

In [37]:
df_playstore=df_playstore.withColumn("Reviews",col("Reviews").cast(IntegerType()))\
                         .withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
                          .withColumn("Installs",col("Installs").cast(IntegerType()))\
                          .withColumn("Price",regexp_replace(col("Price"),"[$]",""))\
                        .withColumn("Price",col("Price").cast(IntegerType()))


In [38]:
df_playstore.printSchema()
df_playstore.show(2,False)

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)

+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
|App                                           |Category      |Rating|Reviews|Installs|Type|Price|Genres                   |
+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
|Photo Editor & Candy Camera & Grid & ScrapBook|ART_AND_DESIGN|4.1   |159    |10000   |Free|0    |Art & Design             |
|Coloring book moana                           |ART_AND_DESIGN|3.9   |967    |500000  |Free|0    |Art & Design;Pretend Play|
+----------------------------------------------+--------------+

In [39]:
df_playstore =df_playstore.na.drop()

In [54]:
df_playstore = df_playstore.filter(~col("Type").isin(["NaN"]) | ~col("Rating").isin(["NaN"]))

In [61]:
df_playstore =df_playstore.filter(~isnan("Rating"))

# Most reviewed Apps 

In [55]:
most_reviewed=df_playstore.groupBy("App").agg(sum("Reviews").alias("TopReviews")).orderBy("TopReviews",ascending=False)

In [56]:
most_reviewed.show(5)

+--------------------+----------+
|                 App|TopReviews|
+--------------------+----------+
|           Instagram| 266241989|
|  WhatsApp Messenger| 207348304|
|      Clash of Clans| 179558781|
|Messenger – Text ...| 169932272|
|      Subway Surfers| 166331958|
+--------------------+----------+
only showing top 5 rows



# Top Downloaded Apps

In [57]:
top_downloads=df_playstore.groupBy("Type").agg(max("Installs").alias("Max_installs")).orderBy("Max_installs",ascending=False)

In [58]:
top_downloads.show()

+----+------------+
|Type|Max_installs|
+----+------------+
|Free|  1000000000|
|Paid|    10000000|
+----+------------+



# Top Rated Apps

In [66]:
top_rated_apps=df_playstore.groupBy("Type","App").agg(max("Rating").alias("highest_ratings")).orderBy("highest_ratings",ascending=False)

In [68]:
top_rated_apps.show(10)

+----+--------------------+---------------+
|Type|                 App|highest_ratings|
+----+--------------------+---------------+
|Free|          AJ RETAILS|            5.0|
|Free|           AP® Guide|            5.0|
|Free|420 BZ Budeze Del...|            5.0|
|Free|       AQ Ria Retail|            5.0|
|Free|      A-Y Collection|            5.0|
|Free|Accounting Quiz (...|            5.0|
|Free|              AJ Cam|            5.0|
|Free|Ag Valley Coopera...|            5.0|
|Free|Anatomy & Physiol...|            5.0|
|Free|American Girls Mo...|            5.0|
+----+--------------------+---------------+
only showing top 10 rows



# Category Preference Analysis

In [80]:
category_pref=df_playstore.groupBy("Category").agg(count("Installs").alias("num_installs"),avg("Rating").alias("avg_rating")).orderBy("num_installs",acending=True)

In [82]:
category_pref.show(4)

+---------+------------+-----------------+
| Category|num_installs|       avg_rating|
+---------+------------+-----------------+
|   BEAUTY|          42|4.278571428571428|
|   EVENTS|          45|4.435555555555557|
|PARENTING|          50|4.300000000000001|
|   COMICS|          58|4.155172413793104|
+---------+------------+-----------------+
only showing top 4 rows



# Genre Exploration with Filtering

In [92]:
#split the genres column to create a new column "Genre"
df_genre=df_playstore.withColumn("Genre",split(col("Genres"),';').getItem(0))

In [93]:
# Filter out genres with fewer than a certain number of apps (e.g., 10)
min_apps_per_genre=10
filtered_genres = df_genre.groupBy("Genre").agg(count("*").alias("num_apps")).filter(col("num_apps") >= min_apps_per_genre)

In [95]:
filtered_genres.show(10)

+---------------+--------+
|          Genre|num_apps|
+---------------+--------+
|      Education|     563|
|         Trivia|      28|
|Auto & Vehicles|      73|
|  Entertainment|     577|
|      Adventure|      89|
|         Arcade|     223|
|         Sports|     337|
| Travel & Local|     226|
|   Food & Drink|     109|
|   Role Playing|     119|
+---------------+--------+
only showing top 10 rows



In [97]:
# Calculate the number of installs and average rating per genre
genre_exploration = df_genre.groupBy("Genre").agg(
    count("Installs").alias("num_installs"),
    avg("Rating").alias("avg_rating")
)

In [99]:
genre_exploration.show(10)

+---------------+------------+------------------+
|          Genre|num_installs|        avg_rating|
+---------------+------------+------------------+
|  Music & Audio|           1|               4.3|
|      Education|         563| 4.312433392539962|
|         Trivia|          28| 4.039285714285714|
|Auto & Vehicles|          73|  4.19041095890411|
|  Entertainment|         577| 4.101386481802426|
|      Adventure|          89| 4.219101123595506|
|         Arcade|         223| 4.308071748878922|
|         Sports|         337| 4.237982195845699|
| Travel & Local|         226|  4.10929203539823|
|   Food & Drink|         109|4.1669724770642205|
+---------------+------------+------------------+
only showing top 10 rows



# Free vs. Paid App Performance

In [102]:
# Filter free and paid apps separately
free_apps = df_playstore.filter(df_playstore["Type"] == "Free")
paid_apps =df_playstore.filter(df_playstore["Type"] == "Paid")

In [107]:
# Calculate average rating and number of installs for free apps
avg_free_rating=free_apps.agg(avg("Rating").alias("avg_free_rating"))
num_installs_free=free_apps.agg(count("Installs").alias("num_installs_free"))    
avg_free_rating.show()
num_installs_free.show()

+-----------------+
|  avg_free_rating|
+-----------------+
|4.186176436847552|
+-----------------+

+-----------------+
|num_installs_free|
+-----------------+
|             8717|
+-----------------+



In [108]:
# Calculate average rating and number of installs for paid apps
avg_paid_rating=paid_apps.agg(avg("Rating").alias("avg_paid_rating"))
num_installs_paid=paid_apps.agg(count("Installs").alias("num_installs_paid"))  
avg_paid_rating.show()
num_installs_paid.show()

+-----------------+
|  avg_paid_rating|
+-----------------+
|4.266615146831529|
+-----------------+

+-----------------+
|num_installs_paid|
+-----------------+
|              647|
+-----------------+



 # Top Categories with High Engagement

In [109]:
# Group by category and calculate average rating and number of installs
category_engagement = df_playstore.groupBy("Category").agg(
    avg("Rating").alias("avg_rating"),
    count("Installs").alias("num_installs")
)

In [113]:
# Filter out categories with low engagement (e.g., less than 10000 installs)
min_installs_threshold = 1000
top_categories = category_engagement.filter(category_engagement["num_installs"] >= min_installs_threshold)

In [114]:
# Sort the top categories by average rating and number of installs
top_categories = top_categories.orderBy(["avg_rating", "num_installs"], ascending=[False, False])
top_categories.show()

+--------+------------------+------------+
|Category|        avg_rating|num_installs|
+--------+------------------+------------+
|    GAME|4.2863263445761195|        1097|
|  FAMILY| 4.192272467086437|        1747|
+--------+------------------+------------+

